In [1]:
import uproot
import ROOT
import numpy as np
import pandas as pd
import math
from collections import OrderedDict
#from coffea import hist


from XRootD import client
from XRootD.client.flags import DirListFlags, StatInfoFlags, OpenFlags, MkDirFlags, QueryCode

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['grid.linestyle'] = ':'

Welcome to JupyROOT 6.12/04


In [2]:
import sys
print(sys.version)

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]


In [3]:
import matplotlib as mpl
mpl.style.use('seaborn-bright')

In [4]:
web_dir = '/publicweb/m/mreid/iDM_AN_Plots/MetTriggerStudyv37'

# Define functions

In [5]:
def extract_teffi(eff):
    _ = ROOT.TCanvas()
    eff.Draw()
    ROOT.gPad.Update()
    
    tot = eff.GetTotalHistogram()
    nBins = tot.GetNbinsX()
    xx = []
    yy = []
    yl = []
    yu = []
    for i in range(1, nBins+1):
        if eff.GetEfficiency(eff.GetGlobalBin(i)) == 0 and eff.GetEfficiencyErrorUp(eff.GetGlobalBin(i)) == 1.: continue
        if eff.GetEfficiency(eff.GetGlobalBin(i)) == 1. and eff.GetEfficiencyErrorLow(eff.GetGlobalBin(i)) == 1.: continue
        
        xx.append(tot.GetXaxis().GetBinCenter(i))
        yy.append(eff.GetEfficiency(eff.GetGlobalBin(i)))
        yl.append(eff.GetEfficiencyErrorLow(eff.GetGlobalBin(i)))
        yu.append(eff.GetEfficiencyErrorUp(eff.GetGlobalBin(i)))
                  
    
    return {'x': np.array(xx),
            'y': np.array(yy),
            'yerr': np.array([yl, yu])}

In [6]:
import scipy.special
from scipy.optimize import curve_fit, fsolve
from scipy import odr

def fsigmoid(z, p0, p1):
    return 1.0 / (1.0 + np.exp(-p0*(z-p1)))
def ferf_cf(z, p0, p1, p2, p3):
    return p2 + p3*scipy.special.erf((z-p0)/p1)
def f98percentile_cf(z, p):
    return ferf_cf(z, *p) - (p[2]+p[3])*0.98
def ferf_odr(p, z):
    return p[2] + p[3]*scipy.special.erf((z-p[0])/p[1])
def f98percentile_odr(z, p):
    return ferf_odr(p, z) - 0.98#(p[2]+p[3])*0.98
def f95percentile_odr(z, p):
    return ferf_odr(p, z) - 0.95#(p[2]+p[3])*0.98

def make_plot(ax, sample, objs, variable, plot_props, event_selection='enrich'):
    h0 = ROOT.TH1F(f'h0_{sample}', '', len(plot_props['axis'])-1, plot_props['axis'])
    h1 = ROOT.TH1F(f'h1_{sample}', '', len(plot_props['axis'])-1, plot_props['axis'])
    for (l1,x),(l2,w) in zip(objs[event_selection][objs[event_selection]['fired0'] == True][variable].iteritems(),objs[event_selection][objs[event_selection]['fired0'] == True]['wgt'].iteritems()):
        h0.Fill(x,abs(w))
    for (l1,x),(l2,w) in zip(objs[event_selection][(objs[event_selection]['fired'] == True) & (objs[event_selection]['fired0'] == True)][variable].iteritems(),objs[event_selection][(objs[event_selection]['fired'] == True) & (objs[event_selection]['fired0'] == True)]['wgt'].iteritems()):
        h1.Fill(x,abs(w))

    eff = ROOT.TEfficiency(h1, h0)
    data = extract_teffi(eff)
    print(data['y'])
    data['x'] = data['x'][((np.isnan(data['yerr'][1])==False))]
    data['y'] = data['y'][((np.isnan(data['yerr'][1])==False))]
    data['yerr'] = [data['yerr'][0][((np.isnan(data['yerr'][1])==False))],data['yerr'][1][((np.isnan(data['yerr'][1])==False))]] 
    #print(data['yerr'][1])
    #data['yerr2'] = [data['yerr'][0][((np.isnan(data['yerr'][1])==False))],data['yerr'][1][((np.isnan(data['yerr'][1])==False))]]
    #data['yerr2'][1][data['yerr2'][1]==0] = 0.1
    #data['yerr2'][0][data['yerr2'][0]==0] = 0.1
    #data['yerr'] = data['yerr2']
    #xerr = np.insert(np.diff(data['x'])/2, 0, (data['x'][1]-data['x'][0])/2)
    xerr = np.diff(plot_props['axis'])/2
    ax.errorbar(x=data['x'], y=data['y'], xerr=xerr, yerr=data['yerr'], 
                label=f'{objs["label"]}', markersize=3, alpha=0.3, fmt='o', c=objs['color'])
    
    ax.set_title(f'Trigger efficiency as a function of MET')
    ax.set_xlabel(plot_props['label'])
    ax.set_ylabel('Trigger efficiency')
    ax.set_ylim((0.9,1.01))
    
    return data
                
def make_plot_scan(ax, sample, objs, variable, plot_props, event_selection='enrich'):
    print(variable)
    h0 = ROOT.TH1F(f'h0_{sample}', '', len(plot_props['axis'])-1, plot_props['axis'])
    h1 = ROOT.TH1F(f'h1_{sample}', '', len(plot_props['axis'])-1, plot_props['axis'])
    for (l1,x),(l2,w) in zip(objs[event_selection][objs[event_selection]['fired0'] == True][variable].iteritems(),objs[event_selection][objs[event_selection]['fired0'] == True]['wgt'].iteritems()):
        h0.Fill(x,w)
    for (l1,x),(l2,w) in zip(objs[event_selection][(objs[event_selection]['fired'] == True) & (objs[event_selection]['fired0'] == True)][variable].iteritems(),objs[event_selection][(objs[event_selection]['fired'] == True) & (objs[event_selection]['fired0'] == True)]['wgt'].iteritems()):
        h1.Fill(x,w)

    eff = ROOT.TEfficiency(h1, h0)
    data = extract_teffi(eff)
    #print(data['y'])
    data['x'] = data['x'][((np.isnan(data['yerr'][1])==False))]
    data['xerr'] = np.diff(plot_props['axis'])/2
    data['xerr'] = data['xerr'][((np.isnan(data['yerr'][1])==False))]
    data['y'] = data['y'][((np.isnan(data['yerr'][1])==False))]
    data['yerr'] = [data['yerr'][0][((np.isnan(data['yerr'][1])==False))],data['yerr'][1][((np.isnan(data['yerr'][1])==False))]] 
    
    return data

def make_plot_fit_cf(ax, sample, objs, variable, plot_props, event_selection='enrich'):
    data = make_plot(ax, sample, objs, variable, plot_props, event_selection)

    minbin = 0 if data['y'][0] < 0.1 else 1
    maxbin = -1
                
    popt, pcov = curve_fit(ferf_cf, data['x'][minbin:maxbin], data['y'][minbin:maxbin], p0=[130,40,1,1],sigma=data['yerr'][1][minbin:maxbin],
                          bounds=([100,30,-10,-10],[250,100,10,10]))
    print(fr"[{props['label']}]")
    print('fit parameter 1-sigma error')
    print('———————————–')
    for i in range(len(popt)):
        print(f'{popt[i]:.2f} +- {pcov[i,i]:.2f}')
    print('')
    
    percentile98 = fsolve(f98percentile_cf, 200, popt)
    fit_range = np.linspace(data['x'][minbin], data['x'][maxbin], 100)
    ax.plot(fit_range, ferf_cf(fit_range, *popt), c=objs['color'],
            label=f'Fit: pT @ 98% = {percentile98[0]:.0f} GeV')
    ax.legend()
    objs[event_selection]['cf_98'] = percentile98[0]
    objs[event_selection]['cf_eff'] = popt[2]+popt[3]  
    
    chi = (data['y'][minbin:maxbin] - ferf_cf(data['x'][minbin:maxbin], *popt)) / data['yerr'][1][minbin:maxbin]
    chi2 = (chi ** 2).sum()
    dof = len(data['x'][minbin:maxbin]) - len(popt)
    objs[event_selection]['cf_chi2'] = (chi2 / dof)
   
          
def make_plot_fit_odr(fit,ax, sample, objs, variable, plot_props, event_selection='enrich'):
    data = make_plot(ax, sample, objs, variable, plot_props, event_selection)

    minbin = 0 #if data['y'][0] < 0.1 else 1
    maxbin = -1
    minbin = 0
    maxbin = -8
    model = odr.Model(ferf_odr)
    xerror = np.repeat(data['x'][1]-data['x'][0], len(data['x'])-minbin)
    real_data = odr.RealData(data['x'][minbin:maxbin], data['y'][minbin:maxbin], sx=xerror, sy=data['yerr'][0][minbin:maxbin])
    #odr_obj = odr.ODR(real_data, model, beta0=[132, 36, 0.5, 0.5])
    odr_obj = odr.ODR(real_data, model, beta0=[75, 50, .5, .5])
    out = odr_obj.run()
    
    popt = out.beta
    perr = out.sd_beta
    print(fr"[{objs['label']}]")
    print('fit parameter 1-sigma error')
    print('———————————–')
    for i in range(len(popt)):
        print(f'{popt[i]:.2f} +- {perr[i]:.2f}')
    print('')
    
    percentile98 = fsolve(f98percentile_odr, 200, popt)
    percentile95 = fsolve(f95percentile_odr, 200, popt)
    if fit:
        fit_range = np.linspace(data['x'][minbin], data['x'][maxbin], 100)
        ax.plot(fit_range, ferf_odr(popt, fit_range), c=objs['color'],
            label=f'Fit: pT @ 98% = {percentile98[0]:.0f} GeV; pT @ 95% = {percentile95[0]:.0f} GeV')#; final eff = {popt[2]+popt[3]:.3f}; chi2 = {out.res_var:.3f}')
        ax.set_title(f'Trigger efficiency as a function of MET (odr fit)')
    
    ax.legend()
    objs[event_selection]['odr_98'] = percentile98[0]
    objs[event_selection]['odr_95'] = percentile98[0]
    objs[event_selection]['odr_eff'] = popt[2]+popt[3]
    objs[event_selection]['odr_chi2'] = out.res_var
    return(data)
          
          
def scan_plot(ax, sample, objs, variable, plot_props, event_selection='enrich',title=''):
    data = make_plot_scan(ax, sample, objs, variable, plot_props, event_selection)

    #minbin = 0 if data['y'][0] < 0.1 else 1
    #maxbin = -1
    per90 =0
    per95 =0
    per98 =0
    pass90 = False
    pass95 = False
    pass98 = False
    for bins in range(len(data['x'])-2):
        #print(bins)
        avg = (data['y'][bins]+data['y'][bins+1]+data['y'][bins+2])/3
        if ((not pass90) and (avg>0.90)): 
            pass90 = True
            per90 = bins+1
        if ((not pass95) and (avg>0.95)): 
            pass95 = True
            per95 = bins+1
        if ((not pass98) and (avg>0.98)): 
            pass98 = True
            per98 = bins+1
            break

    print(fr"90:{data['x'][per90]}:{data['y'][per90]}:{per90}")  
    print(fr"95:{data['x'][per95]}:{data['y'][per95]}:{per95}")
    print(fr"98:{data['x'][per98]}:{data['y'][per98]}:{per98}")
#     if per90 != 0.0:
#         plt.axvline(x=data['x'][per90],linestyle='--',color = objs['color'])
#     if per95 != 0.0:
#         plt.axvline(x=data['x'][per95],linestyle='-.',color = objs['color'])
#     if per98 != 0.0:
#         plt.axvline(x=data['x'][per98],linestyle=':',color = objs['color'])
    thres98= 202
    thres95 = 169
    passtotal_plus = objs[event_selection]['genwgt'].sum()
    pass98_plus = objs[event_selection][(objs[event_selection][variable] >= thres98)]['genwgt'].sum()
    pass95_plus = objs[event_selection][(objs[event_selection][variable] >= thres95)]['genwgt'].sum()
    pass98_plus_trig = objs[event_selection][(objs[event_selection][variable] >= thres98) & (objs[event_selection]['fired'] == True)]['genwgt'].sum()
    pass95_plus_trig = objs[event_selection][(objs[event_selection][variable] >= thres95) & (objs[event_selection]['fired'] == True)]['genwgt'].sum()
    passtotal_plus_trig = objs[event_selection][ (objs[event_selection]['fired'] == True)]['genwgt'].sum()
    print(pass98_plus,pass98_plus_trig)
    eff98 = pass98_plus_trig/pass98_plus
    eff95 = pass95_plus_trig/pass95_plus
    efftot= passtotal_plus_trig/passtotal_plus
    objs['re-label'] = objs['label']# + fr"(95:{eff95:.4f};98:{eff98:.4f})"

    ax.errorbar(x=data['x'], y=data['y'], xerr=data['xerr'], yerr=data['yerr'], 
                label=f'{objs["re-label"]}', markersize=3, fmt='o', c=objs['color'])
    
    ax.set_title(fr"Trigger efficiency as a function of {plot_props['label']} {title}" )
    ax.set_xlabel(plot_props['label'])
    ax.set_ylabel('Trigger efficiency')
    ax.set_ylim((-0.1,1.1))
                   
    ax.legend()
    objs[event_selection]['scan_98'] = data['y'][per98]
    objs[event_selection]['scan_95'] = data['y'][per95]
    objs[event_selection]['scan_90'] = data['y'][per90]
    #objs[event_selection]['odr_chi2'] = out.res_var

# Load data

In [7]:
full_samples = {
    'TT': {
        'label': 'TT',
        'color': 'C1',
        'dir': [
                #'MetTrigStudyv22/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/crab_METTrigEffiv17_TTTo2L2Nu/191012_214113/0000',
                #'MetTrigStudyv22/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/crab_METTrigEffiv17_TTHadronic/191012_214850/0000',
                #'MetTrigStudyv22/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/crab_METTrigEffiv17_TTSemiLeptonic/191012_215307/0000'  
                'MetTrigStudyv23/TTJets_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_TTJets/191017_163301/0000'
                ],
        'xsec': [
                 #364.3,
                 #380.1,
                 #364.3
                831.7600
                ],
        'filename': []
    },
    'WJets': {
        'label': 'WJets',
        'color': 'C2',
        'dir': ['MetTrigStudyv23/WJetsToLNu_HT-70To100_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-70To100/191016_214618/0000',                
                'MetTrigStudyv23/WJetsToLNu_HT-100To200_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-100To200/191016_213058/0000',
                'MetTrigStudyv23/WJetsToLNu_HT-200To400_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-200To400/191016_211244/0000',
                'MetTrigStudyv23/WJetsToLNu_HT-400To600_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-400To600/191017_145239/0000',
                'MetTrigStudyv23/WJetsToLNu_HT-600To800_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-600To800/191016_212903/0000',   
                'MetTrigStudyv23/WJetsToLNu_HT-800To1200_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-800To1200/191017_144932/0000',
                'MetTrigStudyv23/WJetsToLNu_HT-1200To2500_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-1200To2500/191016_211028/0000',
                'MetTrigStudyv23/WJetsToLNu_HT-2500ToInf_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-2500ToInf/191016_213347/0000'
            ],
        'xsec':[1288,##1288.0000,
                1345,#1393.0000,
                359.7,##405.8000,
                48.91,#57.7900,
                12.05,##12.9100,
                5.501,#5.4470,
                1.329,##1.0830,
                0.03216#0.0080     
                ],
        'filename': []
    },
    'DiBoson': {
        'label': 'DiBoson',
        'color': 'C3',
        'dir': [
#                 'MetTrigStudyv22/ZZTo2L2Nu_TuneCP5_13TeV_powheg_pythia8/crab_METTrigEffiv17_ZZTo2L2Nu_ext2/191012_214343/0000',
#                 'MetTrigStudyv22/WZTo3LNu_TuneCP5_13TeV-powheg-pythia8/crab_METTrigEffiv17_WZTo3LNu/191012_220448/0000',
#                 'MetTrigStudyv22/WWTo2L2Nu_NNPDF31_TuneCP5_13TeV-powheg-pythia8/crab_METTrigEffiv17_WWTo2L2Nu/191012_220536/0000',
#                 'MetTrigStudyv21/WWJJToLNuLNu_QCD_noTop_13TeV-madgraph-pythia8/crab_METTrigEffiv16_WWJJToLNuLNu/191004_171252/0000'
                'MetTrigStudyv23/WW_TuneCP5_13TeV-pythia8/crab_METTrigEffiv18_WW/191016_211943/0000',
                'MetTrigStudyv23/WZ_TuneCP5_13TeV-pythia8/crab_METTrigEffiv18_WZ/191016_212811/0000',
                'MetTrigStudyv23/ZZ_TuneCP5_13TeV-pythia8/crab_METTrigEffiv18_ZZ/191016_212342/0000'
                ],
        'xsec':[
                #0.564,#0.6008,
                #4.42965,#4.6580,
                #12.178,#11.08,
                #2.163#2.1630
                118.7000,
                47.2000,
                16.6000
                ],
        'filename': []
    },
    'TriBoson': {
        'label': 'TriBoson',
        'color': 'C4',
        'dir': ['MetTrigStudyv23/WWW_4F_TuneCP5_13TeV-amcatnlo-pythia8/crab_METTrigEffiv18_WWW/191016_213444/0000',
                'MetTrigStudyv23/WWZ_TuneCP5_13TeV-amcatnlo-pythia8/crab_METTrigEffiv18_WWZ/191016_214133/0000',
                'MetTrigStudyv23/WZZ_TuneCP5_13TeV-amcatnlo-pythia8/crab_METTrigEffiv18_WZZ/191017_142237/0000',
                'MetTrigStudyv23/ZZZ_TuneCP5_13TeV-amcatnlo-pythia8/crab_METTrigEffiv18_ZZZ/191016_212955/0000'
        ],
        'xsec':[0.2086,
                0.1651,
                0.05565,
                0.01398,
                ],
        'filename': []
    },
    'SingleT': {
        'label': 'Single Top',
        'color': 'C5',
        'dir': ['MetTrigStudyv23/ST_tW_antitop_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/crab_METTrigEffiv18_tW_antitop/191016_210810/0000',
                'MetTrigStudyv23/ST_tW_top_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/crab_METTrigEffiv18_tW_top/191016_214521/0000',
                'MetTrigStudyv23/ST_s-channel_4f_leptonDecays_TuneCP5_13TeV-madgraph-pythia8/crab_METTrigEffiv18_ST_s-channel/191016_211357/0000',
                'MetTrigStudyv23/ST_t-channel_antitop_5f_TuneCP5_13TeV-powheg-pythia8/crab_METTrigEffiv18_ST_t-channel_antitop/191016_234659/0000',
                'MetTrigStudyv23/ST_t-channel_top_5f_TuneCP5_13TeV-powheg-pythia8/crab_METTrigEffiv18_ST_t-channel_top/191017_142504/0000'
        ],
        'xsec':[35.85,
                35.85,
                3.4000,
                26.2000,
                44.1000
                ],
        'filename': []
    },
    'ZJets': {
        'label': 'ZJets',
        'color': 'C6',
        'dir': ['MetTrigStudyv23/ZJetsToNuNu_HT-100To200_13TeV-madgraph/crab_METTrigEffiv18_ZJetsToNuNu_HT-100To200/191016_211601/0000',
                'MetTrigStudyv23/ZJetsToNuNu_HT-200To400_13TeV-madgraph/crab_METTrigEffiv18_ZJetsToNuNu_HT-200To400/191022_200413/0000',
                'MetTrigStudyv23/ZJetsToNuNu_HT-400To600_13TeV-madgraph/crab_METTrigEffiv18_ZJetsToNuNu_HT-400To600/191016_212037/0000',
                'MetTrigStudyv23/ZJetsToNuNu_HT-600To800_13TeV-madgraph/crab_METTrigEffiv18_ZJetsToNuNu_HT-600To800/191016_212436/0000',
                'MetTrigStudyv23/ZJetsToNuNu_HT-800To1200_13TeV-madgraph/crab_METTrigEffiv18_ZJetsToNuNu_HT-800To1200/191016_211850/0000',
                'MetTrigStudyv23/ZJetsToNuNu_HT-1200To2500_13TeV-madgraph/crab_METTrigEffiv18_ZJetsToNuNu_HT-1200To2500/191016_213742/0000',
                'MetTrigStudyv23/ZJetsToNuNu_HT-2500ToInf_13TeV-madgraph/crab_METTrigEffiv18_ZJetsToNuNu_HT-2500ToInf/191016_214239/0000'
               ],
        'xsec':[280.35,
                77.67,
                10.73,
                2.559,
                1.1796,
                0.28833,
                0.006945
                ],
        'filename': []
    },
       'QCD': {
        'label': 'QCD',
        'color': 'C7',
        'dir': ['MetTrigStudyv23/QCD_bEnriched_HT100to200_TuneCP5_13TeV-madgraph-pythia8/crab_METTrigEffiv18_QCD_bEnriched_HT100to200/191016_212623/0000',
                'MetTrigStudyv23/QCD_bEnriched_HT200to300_TuneCP5_13TeV-madgraph-pythia8/crab_METTrigEffiv18_QCD_bEnriched_HT200to300/191018_133633/0000',
                'MetTrigStudyv23/QCD_bEnriched_HT300to500_TuneCP5_13TeV-madgraph-pythia8/crab_METTrigEffiv18_QCD_bEnriched_HT300to500/191016_212138/0000',
                'MetTrigStudyv23/QCD_bEnriched_HT500to700_TuneCP5_13TeV-madgraph-pythia8/crab_METTrigEffiv18_QCD_bEnriched_HT500to700/191016_212716/0000',
                'MetTrigStudyv23/QCD_bEnriched_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/crab_METTrigEffiv18_QCD_bEnriched_HT700to1000/191016_211506/0000',
                'MetTrigStudyv23/QCD_bEnriched_HT1000to1500_TuneCP5_13TeV-madgraph-pythia8/crab_METTrigEffiv18_QCD_bEnriched_HT1000to1500/191016_211701/0000',
                'MetTrigStudyv23/QCD_bEnriched_HT1500to2000_TuneCP5_13TeV-madgraph-pythia8/crab_METTrigEffiv18_QCD_bEnriched_HT1500to2000/191016_213838/0000',
                'MetTrigStudyv23/QCD_bEnriched_HT2000toInf_TuneCP5_13TeV-madgraph-pythia8/crab_METTrigEffiv18_QCD_bEnriched_HT2000toInf/191016_212242/0000'
        ],
        'xsec':[1122000.0000,
                80170.0000,
                16690.0000,
                1480.0000,
                296.7000,
                46.5000,
                3.7190,
                0.6479,
                ],
        'filename': []
    },
    'DY': {
        'label': 'Drell-Yan',
        'color': 'C8',
        'dir': ['MetTrigStudyv23/DYJetsToLL_M-5to50_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_DYJetsToLL_M-5to50/191016_213542/0000',
                'MetTrigStudyv23/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_DYJetsToLL_M-50toInf/191016_214714/0000'
                ],
        'xsec':[5340.0000,
                66040.0000],
        'filename': []
    },
    'GJets': {
        'label': 'GJets',
        'color': 'C9',
        'dir': [#'MetTrigStudyv23/GJets_HT-40To100_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_GJets_HT-40to100/191017_185015/0000',
                'MetTrigStudyv23/GJets_HT-100To200_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_GJets_HT-100to200/191017_184803/0000',
                'MetTrigStudyv23/GJets_HT-200To400_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_GJets_HT-200to400/191024_131514/0000',
                'MetTrigStudyv23/GJets_HT-400To600_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_GJets_HT-400to600/191017_184913/0000',
                'MetTrigStudyv23/GJets_HT-600ToInf_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_GJets_HT-600toInf/191017_184654/0000'
               ],
        'xsec':[9235,
                2298,
                277.6,
                93.47
                ],
        'filename': []
   }
}


In [8]:
Data_samples = {
     'Data': {
         'label': 'Data',
         'color': 'C4',
         'dir': [
                 'MetTrigStudyv23/SingleMuon/crab_METTrigEffiv18_singleMu_RunA_Sep/191016_210924/0000',
                 'MetTrigStudyv23/SingleMuon/crab_METTrigEffiv18_singleMu_RunB_Sep/191016_214332/0000',
                 'MetTrigStudyv23/SingleMuon/crab_METTrigEffiv18_singleMu_RunC_Sep/191016_214026/0000',
                 'MetTrigStudyv23/SingleMuon/crab_METTrigEffiv18_singleMu_RunD_Jan/191016_213641/0000',
                # 'MetTrigStudyv23/SingleMuon/crab_METTrigEffiv18_singleMu_RunD_Jan/191016_213641/0001'
                 ],
         'xsec':[1./(59.97 *1000),1./(59.97 *1000),1./(59.97 *1000),1./(59.97 *1000)],#,1./(59.97 *1000)],
         'filename': []
     }
}



In [9]:
signal_samples = {
     'M60_l1': {
         'label': 'Mass 60 life 1',
         'color': 'C0',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-60p0_dMchi-20p0_ctau-1'
                 ],
         'xsec':[1],
         'mass': '60x',
         'life': '1x',
         'filename': []
     },
     'M60_l10': {
         'label': 'Mass 60 life 10',
         'color': 'C1',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-60p0_dMchi-20p0_ctau-10'
                 ],
         'xsec':[1],
         'mass': '60x',
         'life': '10x',
         'filename': []
     },
     'M60_l100': {
         'label': 'Mass 60 life 100',
         'color': 'C2',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-60p0_dMchi-20p0_ctau-100'
                 ],
         'xsec':[1],
         'mass': '60x',
         'life': '100x',
         'filename': []
     },
     'M60_l1000': {
         'label': 'Mass 60 life 1000',
         'color': 'C3',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-60p0_dMchi-20p0_ctau-1000'
                 ],
         'xsec':[1],
         'mass': '60x',
         'life': '1000x',
         'filename': []
     },
'M6_l1': {
         'label': 'Mass 6 life 1',
         'color': 'C1',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-6p0_dMchi-2p0_ctau-1'
                 ],
         'xsec':[1],
         'mass': '6x',
         'life': '1x',
         'filename': []
     },
     'M6_l10': {
         'label': 'Mass 6 life 10',
         'color': 'C2',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-6p0_dMchi-2p0_ctau-10'
                 ],
         'xsec':[1],
         'mass': '6x',
         'life': '10x',
         'filename': []
     },
     'M6_l100': {
         'label': 'Mass 6 life 100',
         'color': 'C3',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-6p0_dMchi-2p0_ctau-100'
                 ],
         'xsec':[1],
         'mass': '6x',
         'life': '100x',
         'filename': []
     },
     'M6_l1000': {
         'label': 'Mass 6 life 1000',
         'color': 'C0',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-6p0_dMchi-2p0_ctau-1000'
                 ],
         'xsec':[1],
         'mass': '6x',
         'life': '1000x',
         'filename': []
     },
'M52_l1': {
         'label': 'Mass 52.5 life 1',
         'color': 'C2',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-52p5_dMchi-5p0_ctau-1'
                 ],
         'xsec':[1],
         'mass': '52x',
         'life': '1x',
         'filename': []
     },
     'M52_l10': {
         'label': 'Mass 52.5 life 10',
         'color': 'C3',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-52p5_dMchi-5p0_ctau-10'
                 ],
         'xsec':[1],
         'mass': '52x',
         'life': '10x',
         'filename': []
     },
     'M52_l100': {
         'label': 'Mass 52.5 life 100',
         'color': 'C0',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-52p5_dMchi-5p0_ctau-100'
                 ],
         'xsec':[1],
         'mass': '52x',
         'life': '100x',
         'filename': []
     },
     'M52_l1000': {
         'label': 'Mass 52.5 life 1000',
         'color': 'C1',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-52p5_dMchi-5p0_ctau-1000'
                 ],
         'xsec':[1],
         'mass': '52x',
         'life': '1000x',
         'filename': []
     },
    'M5_l1': {
         'label': 'Mass 5.25 life 1',
         'color': 'C3',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-5p25_dMchi-0p5_ctau-1'
                 ],
         'xsec':[1],
         'mass': '5x',
         'life': '1x',
         'filename': []
     },
     'M5_l10': {
         'label': 'Mass 5.25 life 10',
         'color': 'C0',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-5p25_dMchi-0p5_ctau-10'
                 ],
         'xsec':[1],
         'mass': '5x',
         'life': '10x',
         'filename': []
     },
     'M5_l100': {
         'label': 'Mass 5.25 life 100',
         'color': 'C1',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-5p25_dMchi-0p5_ctau-100'
                 ],
         'xsec':[1],
         'mass': '5x',
         'life': '100x',
         'filename': []
     },
     'M5_l1000': {
         'label': 'Mass 5.25 life 1000',
         'color': 'C2',
         'dir': [
                 'MetTrigStudy_signal_gen_v8/Mchi-5p25_dMchi-0p5_ctau-1000'
                 ],
         'xsec':[1],
         'mass': '5x',
         'life': '1000x',
         'filename': []
     },

}



In [10]:
signal_samples_lifecombo = {
     'M60': {
         'label': 'Mass 50 \u03940.4 GeV',
         'color': 'C0',
         #'dir': [
         #        'MetTrigStudy_signal_gen_v5/Mchi-60p0_dMchi-20p0_ctau-1'
         #        ],
         'xsec':[1],
         'mass': '60x',
         #'life': '1x',
         'filename': []
     },
  
'M6': {
         'label': 'Mass 5 \u03940.4 GeV',
         'color': 'C1',
#          'dir': [
#                  'MetTrigStudy_signal_gen_v5/Mchi-6p0_dMchi-2p0_ctau-1'
#                  ],
         'xsec':[1],
         'mass': '6x',
         #'life': '1x',
         'filename': []
     },
     
'M52': {
         'label': 'Mass 50 \u03940.1 GeV',
         'color': 'C2',
         #'dir': [
         #        'MetTrigStudy_signal_gen_v5/Mchi-52p5_dMchi-5p0_ctau-1'
         #        ],
         'xsec':[1],
         'mass': '52x',
         #'life': '1x',
         'filename': []
     },
    'M5': {
         'label': 'Mass 5 \u03940.1 GeV',
         'color': 'C3',
         #'dir': [
          #       'MetTrigStudy_signal_gen_v5/Mchi-5p25_dMchi-0p5_ctau-1'
         #        ],
         'xsec':[1],
         'mass': '5x',
         #'life': '1x',
         'filename': []
     }

}



In [12]:
xrdfs = client.FileSystem("root://cmseos.fnal.gov/")
redirector = 'root://cmseos.fnal.gov'
basedir = '/store/group/lpcmetx/iDM/Ntuples/2018/signal'
#sample_list = [Di_samples,W_samples,TT_samples]
sample_list = [Data_samples,full_samples]
#sample_list = [Data_samples]
#sample_list = [signal_samples]
#sample_list = [full_samples]
for samples in sample_list:
    for sample, objs in samples.items():
        for sample_index,(xsec, dirs) in enumerate(zip(objs['xsec'],objs['dir'])):
            print(dirs,xsec,sample_index)
    #dirs = objs['dir']
            status, listing = xrdfs.dirlist(f'{basedir}/{dirs}', DirListFlags.STAT)
        #print(listing)
            for file in listing:
                if '.root' in file.name:
                    samples[sample]['filename'].append((f'{redirector}/{basedir}/{dirs}/{file.name}',xsec,sample_index))


MetTrigStudyv23/SingleMuon/crab_METTrigEffiv18_singleMu_RunA_Sep/191016_210924/0000 1.6675004168751043e-05 0
MetTrigStudyv23/SingleMuon/crab_METTrigEffiv18_singleMu_RunB_Sep/191016_214332/0000 1.6675004168751043e-05 1
MetTrigStudyv23/SingleMuon/crab_METTrigEffiv18_singleMu_RunC_Sep/191016_214026/0000 1.6675004168751043e-05 2
MetTrigStudyv23/SingleMuon/crab_METTrigEffiv18_singleMu_RunD_Jan/191016_213641/0000 1.6675004168751043e-05 3
MetTrigStudyv23/TTJets_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_TTJets/191017_163301/0000 831.76 0
MetTrigStudyv23/WJetsToLNu_HT-70To100_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-70To100/191016_214618/0000 1288 0
MetTrigStudyv23/WJetsToLNu_HT-100To200_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-100To200/191016_213058/0000 1345 1
MetTrigStudyv23/WJetsToLNu_HT-200To400_TuneCP5_13TeV-madgraphMLM-pythia8/crab_METTrigEffiv18_WJetsToLNu_HT-200To400/191016_211244/0000 359.7 2
MetTrigStudyv23/WJetsToLNu_

In [ ]:
%%time
luminosity = 59.97 *1000 # 1/fb (femto to pico), 2018
MAX_FILES=None
sample_list = [full_samples,Data_samples]
#sample_list = [Data_samples]
#sample_list = [full_samples]

for samples in sample_list:
    for sample, objs in samples.items():
        objs['enrichx'] = []
        objs['enrich'] = []
       
        print(objs['label'])

        sample_counter=0
        sum_gen_wgt={}
        sum_gen_wgtx={}
        sum_gen_wgt1={}
        sum_gen_wgtx1={}
        file_counter = 1
        max_samples=0
        objs['total_events'] = 0
        current_index = -1
        for filex,xsec,sample_index in objs['filename'][slice(0,MAX_FILES)]:
            if file_counter % 10 == 1:
                total = len(objs['filename'][slice(0,MAX_FILES)])
                print(f'Reading file {file_counter} of {total}')
            file_counter +=1
            if sample_index != current_index:
                objs['total_events'] += xsec*luminosity
                current_index = sample_index
            file = uproot.open(filex)

            objs['trig_treex'] = file['RECO_dsaIso/MetTrigSelfEffiForMuTrack']
            objs['weight_treex'] = file['RECO_dsaIso/GenWeight']

            objs['trig_treex1'] = file['RECO_dsaIsoNoMu/MetTrigSelfEffiForMuTrack']
            objs['weight_treex1'] = file['RECO_dsaIsoNoMu/GenWeight']

            skipx1 = False
            try:
                sum_gen_wgtx1.setdefault(sample_index,0)
                objs['weightx1'] = objs['weight_treex1'].pandas.df(['genwgt'])
                sum_gen_wgtx1[sample_index] += objs['weightx1']['genwgt'].sum()
             
            except:
                print("skipped weightx1: ",filex)
                skipx1 = True
            try:
                if skipx1:
                    print("skippedx1 ")
                    pass
                else:
                    objs['df'] = objs['trig_treex1'].pandas.df()
                    objs['df']['wgt'] = abs(objs['df']['genwgt']*xsec*luminosity)
                    objs['df']['sumwgt_counter'] = sample_index
                    objs['df']['genwgt_sum'] = 0
                    objs['df']['metdiffx'] = objs['df']['recoPFMetPt']*(objs['df']['recoPFMetPhi'].apply(math.cos))-objs['df']['caloMetPt']*(objs['df']['caloMetPhi'].apply(math.cos))
                    objs['df']['metdiffy'] = objs['df']['recoPFMetPt']*(objs['df']['recoPFMetPhi'].apply(math.sin))-objs['df']['caloMetPt']*(objs['df']['caloMetPhi'].apply(math.sin))              
                    objs['df']['metdiff'] =(objs['df']['metdiffx']*objs['df']['metdiffx']+objs['df']['metdiffy']*objs['df']['metdiffy']).apply(math.sqrt)
                    #objs['enrichx'].append(objs['df'][((objs['df']['metdiff']/objs['df']['recoil']) <0.5)&(abs((objs['df']['recoPFMETJetDeltaPhi'])) > 1.57)&(objs['df']['neutralHadronFraction'] < 0.8)&(objs['df']['chargedHadronFraction'] > 0.1)&(objs['df']['pt'].groupby('entry').nth(0) > 40) & (abs(objs['df']['eta'].groupby('entry').nth(0)) < 2.4) & (objs['df']['recoJetPt'] > 120) &(abs(objs['df']['recoJetEta'])<2.4) &(objs['df']['fired0']==1)].groupby('entry').nth(0))                
                    #objs['enrichx'].append(objs['df'][(abs((objs['df']['recoPFMETJetDeltaPhi'])) > 1.57)&(objs['df']['neutralHadronFraction'] < 0.8)&(objs['df']['chargedHadronFraction'] > 0.1)&(objs['df']['pt'].groupby('entry').nth(0) > 40) & (abs(objs['df']['eta'].groupby('entry').nth(0)) < 2.4) & (objs['df']['recoJetPt'] > 120) &(abs(objs['df']['recoJetEta'])<2.4) &(objs['df']['fired0']==1)].groupby('entry').nth(0))                            
                    objs['enrichx'].append(objs['df'][(abs(objs['df']['metdiff']/objs['df']['recoil']) <0.5)&(abs((objs['df']['recoPFrecoilJetDeltaPhi_'])) > 1.57)&(objs['df']['neutralHadronFraction'] < 0.8)&(objs['df']['chargedHadronFraction'] > 0.1)&(objs['df']['pt'].groupby('entry').nth(0) > 40) & (abs(objs['df']['eta'].groupby('entry').nth(0)) < 2.4) & (objs['df']['recoJetPt'] > 120) &(abs(objs['df']['recoJetEta'])<2.4) &(objs['df']['fired0']==1)].groupby('entry').nth(0))                
           
            except:
                print("skipped: ",filex)
                
            skip = False
            try:
                sum_gen_wgt.setdefault(sample_index,0)
                objs['weight'] = objs['weight_treex'].pandas.df(['genwgt'])
                sum_gen_wgt[sample_index] += objs['weight']['genwgt'].sum()
             
            except:
                print("skipped weight: ",filex)
                skip = True
            try:
                if skip:
                    print("skipped ")
                    pass
                else:
                    objs['df'] = objs['trig_treex'].pandas.df()
                    objs['df']['wgt'] = abs(objs['df']['genwgt']*xsec*luminosity)
                    objs['df']['sumwgt_counter'] = sample_index
                    objs['df']['genwgt_sum'] = 0
                    objs['df']['metdiffx'] = objs['df']['recoPFMetPt']*(objs['df']['recoPFMetPhi'].apply(math.cos))-objs['df']['caloMetPt']*(objs['df']['caloMetPhi'].apply(math.cos))
                    objs['df']['metdiffy'] = objs['df']['recoPFMetPt']*(objs['df']['recoPFMetPhi'].apply(math.sin))-objs['df']['caloMetPt']*(objs['df']['caloMetPhi'].apply(math.sin))              
                    objs['df']['metdiff'] =(objs['df']['metdiffx']*objs['df']['metdiffx']+objs['df']['metdiffy']*objs['df']['metdiffy']).apply(math.sqrt)
                    objs['enrich'].append(objs['df'][(abs(objs['df']['metdiff']/objs['df']['recoil']) <0.5)&(abs((objs['df']['recoPFMETJetDeltaPhi'])) > 1.57)&(objs['df']['neutralHadronFraction'] < 0.8)&(objs['df']['chargedHadronFraction'] > 0.1)&(objs['df']['pt'].groupby('entry').nth(0) > 40) & (abs(objs['df']['eta'].groupby('entry').nth(0)) < 2.4) & (objs['df']['recoJetPt'] > 120) &(abs(objs['df']['recoJetEta'])<2.4) &(objs['df']['fired0']==1)].groupby('entry').nth(0))                
                    #objs['enrich'].append(objs['df'].groupby('entry').nth(0))
            except:
                print("skipped: ",filex)

                
            max_samples=sample_index

        objs['enrichx'] = pd.concat(objs['enrichx']).reset_index()
        objs['enrich'] = pd.concat(objs['enrich']).reset_index()
        
        for s_index in range(max_samples+1):
            if 'Data' in objs['label']:
                objs['total_events'] +=sum_gen_wgt[s_index] -1 # -1 to account for xsec
                objs['enrichx']['genwgt_sum']= 1
                objs['enrich']['genwgt_sum'] = 1
            else:
                objs['enrichx']['genwgt_sum'][objs['enrichx']['sumwgt_counter']== s_index] = sum_gen_wgtx1[s_index]
                objs['enrich']['genwgt_sum'][objs['enrich']['sumwgt_counter']== s_index] = sum_gen_wgt[s_index]
            

        objs['enrichx']['wgt'] = abs(objs['enrichx']['wgt']/objs['enrichx']['genwgt_sum'])
        objs['enrich']['wgt'] = abs(objs['enrich']['wgt']/objs['enrich']['genwgt_sum'])


TT
Reading file 1 of 31
Reading file 11 of 31
Reading file 21 of 31
Reading file 31 of 31


/uscms/home/mreid/nobackup/tresenv1/lib/python3.6/site-packages/ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/uscms/home/mreid/nobackup/tresenv1/lib/python3.6/site-packages/ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


WJets
Reading file 1 of 598
Reading file 11 of 598
Reading file 21 of 598
Reading file 31 of 598
Reading file 41 of 598
Reading file 51 of 598
Reading file 61 of 598
Reading file 71 of 598
Reading file 81 of 598
Reading file 91 of 598
Reading file 101 of 598
Reading file 111 of 598
Reading file 121 of 598
Reading file 131 of 598
Reading file 141 of 598
Reading file 151 of 598
Reading file 161 of 598
Reading file 171 of 598
Reading file 181 of 598
Reading file 191 of 598
Reading file 201 of 598
Reading file 211 of 598
Reading file 221 of 598
Reading file 231 of 598
Reading file 241 of 598
Reading file 251 of 598
Reading file 261 of 598
Reading file 271 of 598
Reading file 281 of 598
Reading file 291 of 598
Reading file 301 of 598


In [ ]:
## %%time
luminosity = 59.97 *1000 # 1/fb (femto to pico), 2018
MAX_FILES=None
sample_list = [signal_samples]

for samples in sample_list:
    for sample, objs in samples.items():
        objs['enrich'] = []

       # objs['enrichx'] = []      
        print(objs['label'])

        sample_counter=0
        sum_gen_wgt={}
        sum_gen_wgtx={}
        sum_gen_wgt1={}
        sum_gen_wgtx1={}
        file_counter = 1
        max_samples=0
        for filex,xsec,sample_index in objs['filename'][slice(0,MAX_FILES)]:
            if file_counter % 10 == 1:
                total = len(objs['filename'][slice(0,MAX_FILES)])
                print(f'Reading file {file_counter} of {total}')
            file_counter +=1
        
            file = uproot.open(filex)
            objs['trig_treex'] = file['RECO_dsaIsoNoMu/gen_MetTrigSelfEffiForMuTrack']
            objs['weight_treex'] = file['RECO_dsaIsoNoMu/GenWeight']
            #objs['trig_treex'] = file['RECO_dsaIso/gen_MetTrigSelfEffiForMuTrack']
            #objs['weight_treex'] = file['RECO_dsaIso/GenWeight']

#            objs['trig_treex1'] = file['RECO_dsaIsoNoMu/gen_MetTrigSelfEffiForMuTrack']
#            objs['weight_treex1'] = file['RECO_dsaIsoNoMu/GenWeight']

            skip = False
            skipx = False
            skip1 = False
            skipx1 = False
            skip2 = False
            skipx2 = False
            
            try:
                sum_gen_wgtx.setdefault(sample_index,0)
                objs['weightx'] = objs['weight_treex'].pandas.df(['genwgt'])
                objs['total_eventsx'] = objs['weightx']*xsec*luminosity
                sum_gen_wgtx[sample_index] += objs['weightx']['genwgt'].sum()
             
            except:
                print("skipped weightx: ",filex)
                skipx = True

#             try:
#                 sum_gen_wgtx1.setdefault(sample_index,0)
#                 objs['weightx1'] = objs['weight_treex1'].pandas.df(['genwgt'])
#                 sum_gen_wgtx1[sample_index] += objs['weightx1']['genwgt'].sum()
             
#             except:
#                 print("skipped weightx1: ",filex)
#                 skipx1 = True

            try:
                if skipx:
                    print("skippedx ")
                    pass
                else:
                    objs['df'] = objs['trig_treex'].pandas.df()
                    objs['df']['wgt'] = objs['df']['genwgt']*xsec*luminosity
                    objs['df']['sumwgt_counter'] = sample_index
                    objs['df']['genwgt_sum'] = 0
                    #objs['enrichx'].append(objs['df'][(((objs['df']['recoPFMetPt']-objs['df']['caloMetPt'])/objs['df']['recoil']) <0.5)& (abs((objs['df']['recoPFMETJetDeltaPhi'])) > 0.5)&(objs['df']['neutralHadronFraction'] < 0.8)&(objs['df']['chargedHadronFraction'] > 0.1)&(objs['df']['pt'].groupby('entry').nth(0) > 40) & (abs(objs['df']['eta'].groupby('entry').nth(0)) < 2.4) & (objs['df']['recoJetPt'] > 150) &(abs(objs['df']['recoJetEta'])<2.4) &(objs['df']['fired0']==1)].groupby('entry').nth(0))                
                    objs['enrich'] = objs['df'].groupby('entry').nth(0)#[objs['df']['genJetPt'] > 120]
                   # objs['enrich1'].append(objs['df'][(abs((objs['df']['recoPFMETJetDeltaPhi'])) > 1.57)&(objs['df']['neutralHadronFraction'] < 0.8)&(objs['df']['chargedHadronFraction'] > 0.1)&(objs['df']['pt'].groupby('entry').nth(0) > 40) & (abs(objs['df']['eta'].groupby('entry').nth(0)) < 2.4) & (objs['df']['recoJetPt'] > 120) & (objs['df']['recoJetPt2'] < 30) &(abs(objs['df']['recoJetEta'])<2.4) &(objs['df']['fired0']==1)].groupby('entry').nth(0))                
            except:
                print("skipped: ",filex)
                

#             try:
#                 if skipx1:
#                     print("skippedx1 ")
#                     pass
#                 else:
#                     objs['df'] = objs['trig_treex1'].pandas.df()
#                     objs['df']['wgt'] = objs['df']['genwgt']*xsec*luminosity
#                     objs['df']['sumwgt_counter'] = sample_index
#                     objs['df']['genwgt_sum'] = 0
#                     #objs['enrichx1'].append(objs['df'][(((objs['df']['recoPFMetPt']-objs['df']['caloMetPt'])/objs['df']['recoil']) <0.5)& (abs((objs['df']['recoPFMETJetDeltaPhi'])) > 0.5)&(objs['df']['neutralHadronFraction'] < 0.8)&(objs['df']['chargedHadronFraction'] > 0.1)&(objs['df']['pt'].groupby('entry').nth(0) > 40) & (abs(objs['df']['eta'].groupby('entry').nth(0)) < 2.4) & (objs['df']['recoJetPt'] > 150) &(abs(objs['df']['recoJetEta'])<2.4) &(objs['df']['fired0']==1)].groupby('entry').nth(0))                
#                     objs['enrichx'].append(objs['df'])
#                     # objs['enrichx1'].append(objs['df'][(abs((objs['df']['recoPFMETJetDeltaPhi'])) > 1.57)&(objs['df']['neutralHadronFraction'] < 0.8)&(objs['df']['chargedHadronFraction'] > 0.1)&(objs['df']['pt'].groupby('entry').nth(0) > 40) & (abs(objs['df']['eta'].groupby('entry').nth(0)) < 2.4) & (objs['df']['recoJetPt'] > 120)& (objs['df']['recoJetPt2'] < 30) &(abs(objs['df']['recoJetEta'])<2.4) &(objs['df']['fired0']==1)].groupby('entry').nth(0))                
#             except:
#                 print("skipped: ",filex)


                
            max_samples=sample_index
        #objs['enrich'] = objs['enrich'].reset_index()
#        objs['enrichx'] = pd.concat(objs['enrichx']).reset_index()

        
        for s_index in range(max_samples+1):
            objs['enrich']['genwgt_sum'][objs['enrich']['sumwgt_counter']== s_index] = sum_gen_wgtx[s_index]
#            objs['enrichx']['genwgt_sum'][objs['enrichx']['sumwgt_counter']== s_index] = sum_gen_wgtx1[s_index]

        objs['enrich']['wgt'] = objs['enrich']['wgt']/objs['enrich']['genwgt_sum']
#        objs['enrichx']['wgt'] = objs['enrichx']['wgt']/objs['enrichx']['genwgt_sum']


In [ ]:
signal_samples_lifecombo['M60']['enrich'] = pd.concat([signal_samples['M60_l1']['enrich'],signal_samples['M60_l10']['enrich'],signal_samples['M60_l100']['enrich'],signal_samples['M60_l1000']['enrich']])
signal_samples_lifecombo['M6']['enrich'] = pd.concat([signal_samples['M6_l1']['enrich'],signal_samples['M6_l10']['enrich'],signal_samples['M6_l100']['enrich'],signal_samples['M6_l1000']['enrich']])
signal_samples_lifecombo['M52']['enrich'] = pd.concat([signal_samples['M52_l1']['enrich'],signal_samples['M52_l10']['enrich'],signal_samples['M52_l100']['enrich'],signal_samples['M52_l1000']['enrich']])
signal_samples_lifecombo['M5']['enrich'] = pd.concat([signal_samples['M5_l1']['enrich'],signal_samples['M5_l10']['enrich'],signal_samples['M5_l100']['enrich'],signal_samples['M5_l1000']['enrich']])


In [ ]:
full_samples['TriBoson']['enrichx'][full_samples['TriBoson']['enrichx']['fired']==1][['recoil','recoPFMetPt','wgt']]

In [ ]:
Combo_samples = {
#     'WMuNu': {
#         'label': 'WMuNu (METNoMu)',
#         'color': 'C0'
#     },
    'MC': {
        'label': 'MC (METNoMu)',
        'color': 'C1'
    },
    'Data': {
        'label': 'Data (METNoMu)',
        'color': 'C4'
    },
#     'WMuNuMET': {
#         'label': 'WMuNu (MET)',
#         'color': 'C3'
#     },
    'MCMET': {
        'label': 'MC (MET)',
        'color': 'C0'
    },
    'DataMET': {
        'label': 'Data (MET)',
        'color': 'C2'
    }
}


In [ ]:
Combo_samples['Data']['enrich'] =  Data_samples['Data']['enrichx']
Combo_samples['Data']['total_events'] =  Data_samples['Data']['total_events']
Combo_samples['MC']['enrich'] = pd.concat([full_samples['WJets']['enrichx'],full_samples['ZJets']['enrichx'],full_samples['QCD']['enrichx'],full_samples['DiBoson']['enrichx'],full_samples['TriBoson']['enrichx'],full_samples['SingleT']['enrichx'],full_samples['DY']['enrichx'],full_samples['TT']['enrichx']])#,full_samples['GJets']['enrichx']])
Combo_samples['MC']['total_events'] = sum([full_samples['WJets']['total_events'],full_samples['ZJets']['total_events'],full_samples['QCD']['total_events'],full_samples['DiBoson']['total_events'],full_samples['TriBoson']['total_events'],full_samples['SingleT']['total_events'],full_samples['DY']['total_events'],full_samples['TT']['total_events']])#,full_samples['GJets']['total_events']])

#Combo_samples['WMuNu']['enrichx'] = full_samples['WMuNu']['enrichx']

Combo_samples['DataMET']['enrich'] =  Data_samples['Data']['enrich']
Combo_samples['DataMET']['total_events'] =  Data_samples['Data']['total_events']
#Combo_samples['MCMET']['enrich'] = pd.concat([full_samples['WMuNu']['enrich'],full_samples['WJets']['enrich'],full_samples['DiBoson']['enrich'],full_samples['TT']['enrich']])
#Combo_samples['MCMET']['total_events'] = sum([full_samples['WMuNu']['total_events'],full_samples['WJets']['total_events'],full_samples['DiBoson']['total_events'],full_samples['TT']['total_events']])
Combo_samples['MCMET']['enrich'] = pd.concat([full_samples['WJets']['enrich'],full_samples['ZJets']['enrich'],full_samples['QCD']['enrich'],full_samples['DiBoson']['enrich'],full_samples['TriBoson']['enrich'],full_samples['SingleT']['enrich'],full_samples['DY']['enrich'],full_samples['TT']['enrich']])#,full_samples['GJets']['enrichx']])
Combo_samples['MCMET']['total_events'] = sum([full_samples['WJets']['total_events'],full_samples['ZJets']['total_events'],full_samples['QCD']['total_events'],full_samples['DiBoson']['total_events'],full_samples['TriBoson']['total_events'],full_samples['SingleT']['total_events'],full_samples['DY']['total_events'],full_samples['TT']['total_events']])#,full_samples['GJets']['total_events']])

#Combo_samples['WMuNuMET']['enrichx'] = full_samples['WMuNu']['enrich']


# Comparison of triggered vs pre-selected distributions without Coffea

In [ ]:
eff_plots = OrderedDict({
    'pt': {
        'axis': np.array(np.arange(25,80,3), dtype='float64'),
        'range': [25, 100],
        'label': 'Leading muon pT [GeV]'
    },
    'eta': {
        'axis': np.array(np.linspace(-2.4,2.4,6), dtype='float64'),
        'range': [-2.5, 2.5],
        'label': 'Leading muon eta'
    },
    'phi': {
        'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
        'range': [-3.2, 3.2],
        'label': 'Leading muon phi'
    },
    'recoJetPt': {
        'axis': np.array(np.arange(50,500,50), dtype='float64'),
        'range': [100, 1000],
        'label': 'Leading jet pT [GeV]'
    },
    'recoJetEta': {
        'axis': np.array(np.linspace(-2.4,2.4,6), dtype='float64'),
        'range': [-2.5, 2.5],
        'label': 'Leading jet eta'
    },
    'recoJetPhi': {
        'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
        'range': [-3.2, 3.2],
        'label': 'Leading jet phi'
    },
    'recoPFMetPt': {
      'axis': np.array(np.append(np.append(np.arange(100,200,10),np.arange(200,280,20)),np.arange(280,600,40)), dtype='float64'),        
      #  'axis': np.array(np.append(np.append(np.arange(175,200,5),np.arange(200,280,10)),np.arange(280,500,25)), dtype='float64'),        
        'range': [0, 1200],
        'label': 'MET pT [GeV]'
        #'label': 'recoil (neg) [GeV]'
        #'label': 'recoil [GeV]'
    },
      'recoil': {
        #'axis': np.array(np.append(np.append(np.arange(175,200,5),np.arange(200,280,10)),np.arange(280,500,25)), dtype='float64'),        
        'axis': np.array(np.append(np.append(np.arange(100,200,10),np.arange(200,280,20)),np.arange(280,800,60)), dtype='float64'),        
        'range': [0, 1200],
        #'label': 'MET pT [GeV]'
        #'label': 'recoil (neg) [GeV]'
        'label': 'recoil (MetNoMu) [GeV]'
    },
    'recoPFMetPhi': {
         'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
         'range':[-3.2,3.2],
         'label': 'MET phi'
     },
    'genPFMetSig': {
       #'axis': np.array(np.append(np.append(np.append(np.arange(175,251,3),np.arange(250,481,10)),np.arange(480,611,15)),np.arange(600,1001,30)), dtype='float64'),
        'axis': np.array(np.append(np.append(np.arange(0,180,20),np.arange(180,320,40)),np.arange(320,500,75)), dtype='float64'),
        'range': [0, 450],
        'label': 'signal gen MET pT [GeV]'
    },
})
#     'recoPFMetPhi': {
#         'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
#         'label': 'MET phi'
#     },

In [ ]:
geneff_plots = OrderedDict({
    'genpt': {
        'axis': np.array(np.arange(0,30,5), dtype='float64'),
        'range': [25, 100],
        'label': 'Leading muon pT [GeV]'
    },
    'geneta': {
        'axis': np.array(np.linspace(-2.4,2.4,6), dtype='float64'),
        'range': [-2.5, 2.5],
        'label': 'Leading muon eta'
    },
    'genphi': {
        'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
        'range': [-3.2, 3.2],
        'label': 'Leading muon phi'
    },
    'genJetPt': {
        'axis': np.array(np.arange(50,500,50), dtype='float64'),
        'range': [100, 1000],
        'label': 'Leading jet pT [GeV]'
    },
    'genJetEta': {
        'axis': np.array(np.linspace(-2.4,2.4,6), dtype='float64'),
        'range': [-2.5, 2.5],
        'label': 'Leading jet eta'
    },
    'genJetPhi': {
        'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
        'range': [-3.2, 3.2],
        'label': 'Leading jet phi'
    },
    'genPFMetPt': {
        #'axis': np.array(np.append(np.append(np.arange(175,200,5),np.arange(200,280,10)),np.arange(280,500,25)), dtype='float64'),  
        'axis': np.array(np.arange(25,300,35), dtype='float64'),  
        'range': [0, 450],
        'label': 'MET pT [GeV]'
        #'label': 'recoil (neg) [GeV]'
        #'label': 'recoil [GeV]'
    },
      'genrecoil': {
        #'axis': np.array(np.append(np.append(np.arange(0,200,5),np.arange(200,280,10)),np.arange(280,500,25)), dtype='float64'),  
        'axis': np.array(np.arange(25,300,35), dtype='float64'),
        'range': [0, 450],
        #'label': 'MET pT [GeV]'
        #'label': 'recoil (neg) [GeV]'
        'label': 'recoil (MetNoMu) [GeV]'
    },
#     'genPFMetSig': {
#        #'axis': np.array(np.append(np.append(np.append(np.arange(175,251,3),np.arange(250,481,10)),np.arange(480,611,15)),np.arange(600,1001,30)), dtype='float64'),
#         'axis': np.array(np.append(np.append(np.arange(0,180,20),np.arange(180,320,40)),np.arange(320,500,75)), dtype='float64'),
#         'range': [0, 450],
#         'label': 'signal gen MET pT [GeV]'
#     },
#     'genPFMetPhi': {
#          'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
#          'range':[-3.2,3.2],
#          'label': 'MET phi'
#      },
})
#     'recoPFMetPhi': {
#         'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
#         'label': 'MET phi'
#     },

In [ ]:
sample= 'Data'
#sample= 'MC'
#samples = full_samples
samples = Combo_samples

fig, axes = plt.subplots(3,3, figsize=(10,6))
fig.suptitle(f'Triggered events vs pre-selected events ({samples[sample]["label"]})')
plt.tight_layout(h_pad=2.0)
fig.subplots_adjust(top=0.90)
#axes[2,1].axis('off')
#axes[2,2].axis('off')
log = True
for index, (column, props) in enumerate(eff_plots.items()):
    if column is 'genPFMetSig':
        continue
    ax = axes[index//3, index%3]
    bin_edges = props['axis']
    kwargs = {'range': props['range'], 'bins':20, 'density': False, 'histtype':'step', 'log':log}
    
    df = samples[sample]['enrich']
    ax.hist(df[(df['fired0']==1)][column], **kwargs, weights=df[(df['fired0']==1)]['wgt'],
            label='Pre-selected events: passed single mu trig');
    ax.hist(df[(df['fired0']==1) & (df['fired']==1)][column], **kwargs, weights=df[(df['fired0']==1) & (df['fired']==1)]['wgt'],
            label='Pre-selected && triggered events');
    ax.set_xlabel(props['label'])
    ax.set_ylabel('A. U.')

ax.legend(loc=(1.2 ,0.4))
fig.savefig(f'{web_dir}/Trig_vs_preselected_{sample}_log_{log}_againstrecoil.png', bbox_inches='tight')


In [ ]:
#sample = 'DiBoson'
#sample = 'WMuNu'
#sample = 'TT'
#sample = 'WJets'
sample = 'Data'
#sample= 'MC'
#samples = full_samples
samples = Combo_samples

fig, ax = plt.subplots( figsize=(10,6))
fig.suptitle(f'Met vs MetNoMu ({samples[sample]["label"]})')
plt.tight_layout(h_pad=2.0)
fig.subplots_adjust(top=0.90)
#axes[2,1].axis('off')
#axes[2,2].axis('off')

#for index, (column, props) in enumerate(eff_plots.items()):
    #ax = axes[index//3, index%3]
    
#    bin_edges = props['axis']
kwargs = {'bins':[100,100],'range':[[0,500],[0,500]]}
    
df = samples[sample]['enrichx']
    #print(df)
    #x.hist(df.query('fired0==True')[column].groupby('entry').nth(0), **kwargs, 
    #       label='Pre-selected events: passed single mu trig');
    #x.hist(df.query('fired0==True & fired==True')[column].groupby('entry').nth(0), **kwargs, 
    #       label='Pre-selected && triggered events');
    #print(df[['fired0','fired','recoPFMetPt']].groupby('entry').nth(0))
    #print(len(df[df['fired0']==True].groupby('entry').nth(0)))
    #print(len(df[['fired0','fired','recoPFMetPt']].groupby('entry').nth(0)))
h=ax.hist2d(df['recoPFMetPt'],df['recoil'], **kwargs, weights=df['wgt'],
            label='Pre-selected events: passed single mu trig');
    #ax.hist2d(df[(df['fired0']==1)][column], **kwargs, weights=df[(df['fired0']==1)]['wgt'],
    #        label='Pre-selected events: passed single mu trig');
    #ax.hist(df[(df['fired0']==1) & (df['fired']==1)][column], **kwargs, weights=df[(df['fired0']==1) & (df['fired']==1)]['wgt'],
    #        label='Pre-selected && triggered events');
ax.set_xlabel("recoPFMetPt [Gev]")
ax.set_ylabel("recoil [Gev]")
ax.legend()
plt.colorbar(h[3], ax=ax)
#ax.set_ylabel('A. U.')

#ax.legend(loc=(1.2 ,0.4))
fig.savefig(f'{web_dir}/Met_vs_MetNoMu_{sample}.png', bbox_inches='tight')


# Trigger efficiency of `MET_120` vs kinematic variables -- pre-selected events

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(10,6))
fig.suptitle('trigger efficiency (triggered events / pre-selected events)')
plt.tight_layout(h_pad=2.0)
fig.subplots_adjust(top=0.90)
#axes[2,1].axis('off')
axes[2,2].axis('off')
#samples = Combo_samples
samples = signal_samples_lifecombo
#samples = full_samples
for index, (column, properties) in enumerate(geneff_plots.items()):
    ax = axes[index//3, index%3]
    
    bin_edges = properties['axis']
    
    for sample, objs in samples.items():
        #if objs['label'] is not 'WJets':
        #    continue
        df = samples[sample]['enrich']#.groupby('entry').nth(0)
        if 'h0' in locals():
            del h0,h1
    
        h0 = ROOT.TH1F(f'h0_1', '', len(bin_edges)-1, bin_edges)
        h1 = ROOT.TH1F(f'h1_1', '', len(bin_edges)-1, bin_edges)

        #or l,x in objs['df'].query('fired0==True')[column].groupby('entry').nth(0).iteritems():
        #   h0.Fill(x)
        #or l,x in objs['df'].query('fired0==True & fired==True')[column].groupby('entry').nth(0).iteritems():
        #   h1.Fill(x)
            
        for (l1,x),(l2,w) in zip(df[df['fired0']==True][column].iteritems(),df[df['fired0']==True]['wgt'].iteritems()):
            h0.Fill(x,w)
        for (l1,x),(l2,w) in zip(df[(df['fired0']==True) & (df['fired']==True)][column].iteritems(),df[(df['fired0']==True) & (df['fired']==True)]['wgt'].iteritems()):
            h1.Fill(x,w)

        eff = ROOT.TEfficiency(h1, h0)
        data = extract_teffi(eff)

        #print(data['x'])
        if 'Pt' in column:# 'pt':
            xerr=np.diff(bin_edges)/2
        else:
            xerr = np.insert(np.diff(data['x'])/2, 0, (data['x'][1]-data['x'][0])/2)
        if len(data['x']) != len(properties['axis'])-1:
            recover = np.add(np.diff(properties['axis'])/2,properties['axis'][:-1])
            data['x']= np.append(data['x'],recover[len(data['x']):])
            data['y']= np.append(data['y'],np.full(len(recover)-len(data['y']),0))
            data['yerr']= np.append(data['yerr'],np.full((2,len(recover)-len(data['yerr'][0])),0),axis=1)
        #print(xerr)
        #print(data['x'])
        ax.errorbar(x=data['x'], y=data['y'], xerr=xerr, yerr=data['yerr'],
                    fmt='o', label=f'{objs["label"]}', ms=3, c=objs['color'])
        ax.set_xlabel(properties['label'])
        ax.set_ylabel('Trigger efficiency')
        ax.set_ylim(0,1.1)
        
handles, labels = ax.get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc=(-0.8,0.1));

fig.savefig(f'{web_dir}/trigger_eff_preselected_signalMET.png', bbox_inches='tight')

# DATA and MC Trigger efficiency 

In [ ]:
# Using cf (curve_fit), no x or y errors in the fit

residuals = False
if residuals:
    sels = ['enrich']
    com = 'MC'
else:
    sels = ['enrich']
    com = 'sample'
values = {}
xtype='recoPFMetPt'
#xtype='recoil'
fit=True
for sel in sels:
    if residuals:
        fig, (ax,ax1) = plt.subplots(2,1,gridspec_kw={'height_ratios': [3,1]},figsize=(8,6))
        samples = Combo_samples
        #sel = 'enrich'
    else:
        fig, (ax,ax1) = plt.subplots(2,1,figsize=(8,6))
        samples = full_samples
        #sels = ['enrich','enrichx']
    plt.tight_layout()
#fig.subplots_adjust(top=0.90)
#for sel in sels:
    #ax = axes[0]
    for index, (sample, props) in enumerate(samples.items()):
        if "GJets" in props['label']:
            print("Skipping: ",props['label'])
            continue
#         if "(MET)" in props['label']:
#             print("Skipping: ",props['label'])
#             continue
        #try:
        if not residuals and index >=4:
            values[props['label']] = make_plot_fit_odr(fit,ax1, sample, props, xtype, eff_plots[xtype], event_selection=sel)
            ax1.legend(loc=(1.2 ,0.))
        else:
            values[props['label']] = make_plot_fit_odr(fit, ax, sample, props, xtype, eff_plots[xtype], event_selection=sel)
            ax.legend(loc=(1.2 ,0.))
        #except:
        #    print("Skipping: ",props['label'])
        #    continue

        
        
    if(residuals):
        residualx = []
        residualy = []
        residualerr = []
        for i, (datax,datay,dataerr) in enumerate(zip(values['Data (METNoMu)']['x'],values['Data (METNoMu)']['y'],values['Data (METNoMu)']['yerr'][1])):
            for j, (MCx,MCy,MCerr) in enumerate(zip(values['MC (METNoMu)']['x'],values['MC (METNoMu)']['y'],values['MC (METNoMu)']['yerr'][1])):
                if MCx == datax:
                    residualx.append(MCx)
                    residualy.append(datay/MCy)
                    residualerr.append((datay/MCy)*((MCerr/MCy)**2 + (dataerr/datay)**2)**(0.5))
        
        residualx_met = []
        residualy_met = []
        residualerr_met = []
        for i, (datax,datay,dataerr) in enumerate(zip(values['Data (MET)']['x'],values['Data (MET)']['y'],values['Data (MET)']['yerr'][1])):
            for j, (MCx,MCy,MCerr) in enumerate(zip(values['MC (MET)']['x'],values['MC (MET)']['y'],values['MC (MET)']['yerr'][1])):
                if MCx == datax:
                    residualx_met.append(MCx)
                    residualy_met.append(datay/MCy)
                    residualerr_met.append((datay/MCy)*((MCerr/MCy)**2 + (dataerr/datay)**2)**(0.5))
                    
        
        ax1.errorbar(x=residualx, y=residualy, yerr=residualerr, 
                        label=f'Data/MC MetNoMu', markersize=3, alpha=0.3, fmt='o')
        ax1.errorbar(x=residualx_met, y=residualy_met, yerr=residualerr_met, 
                       label=f'Data/MC Met', markersize=3, alpha=0.3, fmt='o',color='r')
        ax1.legend()
        ax1.set_xlabel(eff_plots[xtype]['label'])
        ax1.set_ylabel('Data/MC')
        plt.axhline(y=1.00,linestyle='--',color = 'r')
        ax1.set_ylim(0.93,1.02)
        ax1.yaxis.set_ticks([0.94,0.96,0.98,1.0,1.02,1.04])
    else:
        ax1.set_ylim(0.9,1.01)
    
    if xtype == 'recoil':
        ax.set_ylim(0.8,1.01)
    else:
        ax.set_ylim(0.8,1.01)
    mettype = {'enrich':'Met','enrichx':'MetNoMu'}
    fig.savefig(f'{web_dir}/scan_for_threshold_MetNoMuStudy_{xtype}_{com}_{mettype[sel]}.png', bbox_inches='tight')

In [ ]:
# Using cf (curve_fit), no x or y errors in the fit

residuals = True
if residuals:
    sels = ['enrichx']
    com = 'MC'
else:
    sels = ['enrich','enrichx']
    com = 'sample'
values = {}
xtype='recoPFMetPt'
#xtype='recoil'
fit=True
for sel in sels:
    data_props = Data_samples['Data']
    for index, (sample, props) in enumerate(samples.items()):

        fig, (ax,ax1) = plt.subplots(2,1,gridspec_kw={'height_ratios': [3,1]},figsize=(8,6))
        plt.tight_layout()

        values[props['label']] = make_plot_fit_odr(fit, ax, sample, props, xtype, eff_plots[xtype], event_selection=sel)
        values['Data'] = make_plot_fit_odr(fit, ax, 'Data', data_props, xtype, eff_plots[xtype], event_selection=sel)
        ax.legend(loc=(1.2 ,0.))

        residualx = []
        residualy = []
        residualerr = []
        for i, (datax,datay,dataerr) in enumerate(zip(values['Data']['x'],values['Data']['y'],values['Data']['yerr'][1])):
            for j, (MCx,MCy,MCerr) in enumerate(zip(values[props['label']]['x'],values[props['label']]['y'],values[props['label']]['yerr'][1])):
                if MCx == datax:
                    residualx.append(MCx)
                    residualy.append(datay/MCy)
                    residualerr.append((datay/MCy)*((MCerr/MCy)**2 + (dataerr/datay)**2)**(0.5))
        
        
        ax1.errorbar(x=residualx, y=residualy, yerr=residualerr, 
                        label=f'Data/MC', markersize=3, alpha=0.3, fmt='o')
        ax1.legend()
        ax1.set_xlabel(eff_plots[xtype]['label'])
        ax1.set_ylabel('Data/MC')
        plt.axhline(y=1.00,linestyle='--',color = 'r')
        ax1.set_ylim(0.93,1.02)
        ax1.yaxis.set_ticks([0.94,0.96,0.98,1.0,1.02,1.04])

        mettype = {'enrich':'Met','enrichx':'MetNoMu'}
        fig.savefig(f'{web_dir}/scan_for_threshold_MetNoMuStudy_{xtype}_{sample}_{mettype[sel]}.png', bbox_inches='tight')

In [ ]:
tablesMET=[[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
colsMET = []
for index, (sample, props) in enumerate(full_samples.items()):
    colsMET.append(props['label'])
    tablesMET[0].append(props['total_events'])
    tablesMET[1].append(props['enrich']['wgt'].sum())
    tablesMET[2].append(props['enrich'][props['enrich']['fired']==1]['wgt'].sum()),
    tablesMET[3].append(props['enrichx'][props['enrichx']['fired']==1]['wgt'].sum()),
    tablesMET[4].append(float(props['enrich'][props['enrich']['fired']==1]['wgt'].sum())/props['enrich']['wgt'].sum())
    tablesMET[5].append(float(props['enrichx'][props['enrichx']['fired']==1]['wgt'].sum())/props['enrichx']['wgt'].sum())
    tablesMET[6].append(float(props['enrich'][props['enrich']['fired']==1]['wgt'].sum())/props['total_events'])
    tablesMET[7].append(float(props['enrichx'][props['enrichx']['fired']==1]['wgt'].sum())/props['total_events'])
    tablesMET[8].append(float(props['enrich'][(props['enrich']['fired']==1) & (props['enrich']['recoPFMetPt']>200)]['wgt'].sum())/props['enrich'][props['enrich']['recoPFMetPt']>200]['wgt'].sum())
    tablesMET[9].append(float(props['enrichx'][(props['enrichx']['fired']==1) & (props['enrichx']['recoPFMetPt']>200)]['wgt'].sum())/props['enrichx'][props['enrichx']['recoPFMetPt']>200]['wgt'].sum())
    tablesMET[10].append(props['enrichx']['odr_98'].iloc[0])
    tablesMET[11].append(props['enrichx']['odr_95'].iloc[0])
    tablesMET[12].append(props['enrichx']['odr_eff'].iloc[0])
    tablesMET[13].append(props['enrichx']['odr_chi2'].iloc[0])

tables=[[],[],[],[],[],[],[],[],[],[]]
cols = []
for index, (sample, props) in enumerate(Combo_samples.items()):
    cols.append(props['label'])
    tables[0].append(props['total_events'])
    tables[1].append(props['enrich']['wgt'].sum())
    tables[2].append(props['enrich'][props['enrich']['fired']==1]['wgt'].sum()),
    tables[3].append(float(props['enrich'][props['enrich']['fired']==1]['wgt'].sum())/props['enrich']['wgt'].sum())
    tables[4].append(float(props['enrich'][props['enrich']['fired']==1]['wgt'].sum())/props['total_events'])
    tables[5].append(float(props['enrich'][(props['enrich']['fired']==1) & (props['enrich']['recoPFMetPt']>200)]['wgt'].sum())/props['enrich'][props['enrich']['recoPFMetPt']>200]['wgt'].sum())
    tables[6].append(props['enrich']['odr_98'].iloc[0])
    tables[7].append(props['enrich']['odr_95'].iloc[0])
    tables[8].append(props['enrich']['odr_eff'].iloc[0])
    tables[9].append(props['enrich']['odr_chi2'].iloc[0])
beamMET = pd.DataFrame(tablesMET, columns=colsMET,index=['total events', 'selection pass', 'sel+METtrigger pass' , 'sel + MetNoMutrigger pass' ,'MET trig eff (selection)' ,'MetNoMu trig eff (selection)' , 'MET trig eff (total)','MetNoMu trig eff (total)','MET trig eff (sel Met>200)','MetNoMu trig eff (sel Met>200)','odr 98','odr 95','eff','chi2'])
print(beamMET)
beam = pd.DataFrame(tables, columns=cols,index=['total events', 'selection pass', 'sel+trigger pass' , 'trig eff (selection)' , 'trig eff (total)','trig eff (sel Met>200)','odr 98','odr 95','eff','chi2'])
print(beam)

In [ ]:
# Using cf (curve_fit), no x or y errors in the fit
#samples = full_samples
samples = Combo_samples
#samples=Comp_Data
sels = ['enrichx']#,'enrichx']
for sel in sels:
    fig, ax = plt.subplots(figsize=(8,6))
    for index, (sample, props) in enumerate(samples.items()):
        #if "recoil" not in props['label'] or "3rd" in props['label'] or "jetcut" in props['label'] :
         #   continue
        scan_plot(ax, sample, props, 'recoPFMetPt', eff_plots['recoPFMetPt'], event_selection=sel)
        #scan_plot(ax, sample, props, 'recoil', eff_plots['recoilPt'], event_selection=sel)
        #make_plot_fit_odr(ax, sample, props, 'recoPFMetPt', eff_plots['recoPFMetPt'], event_selection=sel)
    fig.savefig(f'{web_dir}/scan_for_threshold_MetNoMuStudy_full.png', bbox_inches='tight')



# Signal Efficiency

In [ ]:
# Using cf (curve_fit), no x or y errors in the fit
#samples = full_samples
#samples = Combo_samples
samples=signal_samples
sel = 'enrich'
masses = ['60x','52x','6x','5x']
lifetimes = ['1x','10x','100x','1000x']
mass_label = {'60x':'50 \u03940.4 GeV','52x':'50 \u03940.1 GeV','6x':'5 \u03940.4 GeV','5x':'5 \u03940.1 GeV'}
for mass in masses:
    fig, ax = plt.subplots(figsize=(8,6))
    for index, (sample, props) in enumerate(samples.items()):
        if mass not in props['mass']:# or '1000' in props['life']:
            continue
        scan_plot(ax, sample, props, 'genPFMetPt', eff_plots['genPFMetSig'], event_selection=sel, title=fr'mass = {mass_label[mass]}')
    fig.savefig(f'{web_dir}/signal_eff_{mass}_masked.png', bbox_inches='tight')



In [ ]:
# Using cf (curve_fit), no x or y errors in the fit
#samples = full_samples
#samples = Combo_samples
samples=signal_samples
sel = 'enrich'
masses = ['60x','52x','6x','5x']
lifetimes = ['1x','10x','100x','1000x']
life_label = {'1x':'1mm','10x':'10mm','100x':'100mm','1000x':'1000mm'}
for life in lifetimes:
    fig, ax = plt.subplots(figsize=(8,6))
    for index, (sample, props) in enumerate(samples.items()):
        if life not in props['life']:# or '1000' in props['life']:
            continue
        scan_plot(ax, sample, props, 'genPFMetPt', eff_plots['genPFMetSig'], event_selection=sel, title=fr'lifetime = {life_label[life]}')
    fig.savefig(f'{web_dir}/signal_eff_{life}_masked.png', bbox_inches='tight')



In [ ]:
print(signal_samples_lifecombo['M60']['enrich'])

In [ ]:
# Using cf (curve_fit), no x or y errors in the fit
#samples = full_samples
#samples = Combo_samples
samples=signal_samples_lifecombo
sel = 'enrich'
masses = ['60x','52x','6x','5x']
lifetimes = ['1x','10x','100x','1000x']
mass_label = {'60x':'50 \u03940.4 GeV','52x':'50 \u03940.1 GeV','6x':'5 \u03940.4 GeV','5x':'5 \u03940.1 GeV'}
fig, ax = plt.subplots(figsize=(8,6))
for index, (sample, props) in enumerate(samples.items()):
        #if life not in props['life']:# or '1000' in props['life']:
         #   continue
    #scan_plot(ax, sample, props, 'genPFMetPt', eff_plots['genPFMetSig'], event_selection=sel)#, title=fr'lifetime = {mass_label[mass]}')
    scan_plot(ax, sample, props, 'genrecoil', eff_plots['genrecoil'], event_selection=sel)#, title=fr'lifetime = {mass_label[mass]}')
#fig.savefig(f'{web_dir}/signal_eff_all_masked_METNoMu.png', bbox_inches='tight')
fig.savefig(f'{web_dir}/signal_eff_all_masked_METNoMu_recoil.png', bbox_inches='tight')



In [ ]:
samples=signal_samples_lifecombo
totals = {'60x':{},'6x':{},'52x':{},'5x':{}}
passes = {'60x':{},'6x':{},'52x':{},'5x':{}}
totals200 = {'60x':{},'6x':{},'52x':{},'5x':{}}
passes200 = {'60x':{},'6x':{},'52x':{},'5x':{}}
masses = ['60x','52x','6x','5x']
lifetimes = ['1x','10x','100x','1000x']
for mass in masses:
    #for life in lifetimes:
    for index, (sample, props) in enumerate(samples.items()):
        totals[props['mass']] = props['enrich']['genwgt'].sum()
        passes[props['mass']] = props['enrich'][props['enrich']['fired']==1]['genwgt'].sum()
        totals200[props['mass']] = props['enrich'][props['enrich']['genPFMetPt']> 200]['genwgt'].sum()
        passes200[props['mass']] = props['enrich'][(props['enrich']['genPFMetPt']> 200) & (props['enrich']['fired']==1)]['genwgt'].sum()
print(totals)
print(passes)
print(totals200)
print(passes200)
tables = {}
beam = {}
table = [
    [totals['60x'],totals['52x'],totals['6x'],totals['5x']],
    [passes['60x'],passes['52x'],passes['6x'],passes['5x']],
    [passes['60x']/totals['60x'],passes['52x']/totals['52x'],passes['6x']/totals['6x'],passes['5x']/totals['5x']],
    [totals200['60x'],totals200['52x'],totals200['6x'],totals200['5x']],
    [passes200['60x'],passes200['52x'],passes200['6x'],passes200['5x']],
    [passes200['60x']/totals200['60x'],passes200['52x']/totals200['52x'],passes200['6x']/totals200['6x'],passes200['5x']/totals200['5x']]]
    
beam = pd.DataFrame(table,columns=['50 \u03940.4 GeV','50 \u03940.1 GeV','5 \u03940.4 GeV','5 \u03940.1 GeV'], index=['total','pass','total efficiency','total (Met>200)','pass (Met>200)','efficiency (Met>200)'])
print(beam)
# for mass in masses:
#     tables[mass]=[[totals[mass]['1x'],totals[mass]['10x'],totals[mass]['100x'],totals[mass]['1000x']],
#          [passes[mass]['1x'],passes[mass]['10x'],passes[mass]['100x'],passes[mass]['1000x']],
#           [float(passes[mass]['1x']/totals[mass]['1x']),float(passes[mass]['10x']/totals[mass]['10x']),float(passes[mass]['100x']/totals[mass]['100x']),float(passes[mass]['1000x']/totals[mass]['1000x'])]]#,
#     beam[mass] = pd.DataFrame(tables[mass], columns=['1','10','100','1000'])
# #print(beam['60x'])
# #print(totals['60x']['1x'])
# print(beam['60x'])
# print(beam['52x'])
# print(beam['6x'])
# print(beam['5x'])

In [ ]:
samples=signal_samples
totals = {'60x':{},'6x':{},'52x':{},'5x':{}}
passes = {'60x':{},'6x':{},'52x':{},'5x':{}}
masses = ['60x','52x','6x','5x']
lifetimes = ['1x','10x','100x','1000x']
for mass in masses:
    for life in lifetimes:
        for index, (sample, props) in enumerate(samples.items()):
            totals[props['mass']][props['life']] = props['enrich']['genwgt'].sum()
            passes[props['mass']][props['life']] = props['enrich'][props['enrich']['fired']==1]['genwgt'].sum()
print(totals)
print(passes)
tables = {}
beam = {}
for mass in masses:
    tables[mass]=[[totals[mass]['1x'],totals[mass]['10x'],totals[mass]['100x'],totals[mass]['1000x']],
          [passes[mass]['1x'],passes[mass]['10x'],passes[mass]['100x'],passes[mass]['1000x']],
          [float(passes[mass]['1x']/totals[mass]['1x']),float(passes[mass]['10x']/totals[mass]['10x']),float(passes[mass]['100x']/totals[mass]['100x']),float(passes[mass]['1000x']/totals[mass]['1000x'])]]#,
    beam[mass] = pd.DataFrame(tables[mass], columns=['1','10','100','1000'])
#print(beam['60x'])
#print(totals['60x']['1x'])
print(beam['60x'])
print(beam['52x'])
print(beam['6x'])
print(beam['5x'])

# OLD EFFICIENCY PLOTTING

In [ ]:
# Using cf (curve_fit), no x or y errors in the fit
#samples = full_samples
samples = Combo_samples
#samples = TT_samples
#samples = W_samples
#samples = Di_samples
sels = ['enrich','enrich1','enrichx','enrich1x']
sel_label = {'enrich':'200_x25','enrich1':'200_x50','enrichx':'200_x25iso','enrich1x':'200_x50iso'}
for sel in sels:
    fig, ax = plt.subplots(figsize=(8,6))
    for index, (sample, props) in enumerate(samples.items()):
        if props['label'] is 'TT':
            continue
        #make_plot_fit_cf(ax, sample, props, 'recoPFMetPt', eff_plots['recoPFMetPt'], event_selection='enrich1x')
        make_plot_fit_cf(ax, sample, props, 'recoPFMetPt', eff_plots['recoPFMetPt'], event_selection=sel)
    #fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_full_{sel_label[sel]}.png', bbox_inches='tight')
    fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_combo_{sel_label[sel]}.png', bbox_inches='tight')



#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_TT.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_W.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_diboson.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_data.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_full_200_x25.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_full_200_x50.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_full_200_x25iso.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_full_200_x50iso.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_combo_200_x25.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_combo_200_x50.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_combo_200_x25iso.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_cf_combo_200_x50iso.png', bbox_inches='tight')

In [ ]:
# Using ODR, includes both x errors (bin width) and y errors
#samples = full_samples
samples = Combo_samples
#samples = TT_samples
#samples = W_samples
#samples = Di_samples
sels = ['enrich','enrich1','enrichx','enrich1x']
sel_label = {'enrich':'200_x25','enrich1':'200_x50','enrichx':'200_x25iso','enrich1x':'200_x50iso'}
for sel in sels:
    fig, ax = plt.subplots(figsize=(8,6))
    for index, (sample, props) in enumerate(samples.items()):
        if props['label'] is 'TT':
            continue
        make_plot_fit_odr(ax, sample, props, 'recoPFMetPt', eff_plots['recoPFMetPt'], event_selection=sel)
        #make_plot_fit_odr(ax, sample, props, 'recoPFMetPt', eff_plots['recoPFMetPt'], event_selection='enrich1x')
    #fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_full_200_{sel_label[sel]}.png', bbox_inches='tight')
    fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_combo_200_{sel_label[sel]}.png', bbox_inches='tight')



#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_TT.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_W.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_diboson.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_data.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_full_200_x25.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_full_200_x50.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_full_200_x25iso.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_full_200_x50iso.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_combo_200_x25.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_combo_200_x50.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_combo_200_x25iso.png', bbox_inches='tight')
#fig.savefig(f'{web_dir}/trigger_vs_met_fit_preselected_ODR_combo_200_x50iso.png', bbox_inches='tight')

In [ ]:
for index, (sample, props) in enumerate(full_samples.items()):
    if props['label'] == 'TT':
        continue
    print(props)

# Trigger efficiency of `MET_120`  vs kinematic distributions -- all events

In [ ]:
#samples= full_samples
#info = pd.Dataframe()
sels = ['enrich','enrich1','enrichx','enrich1x']
sel_label= {'enrich': 'mu50trig, mu>25','enrich1': 'mu50trig, mu>50','enrichx': 'isomu24trig, mu>25','enrich1x': 'isomu24trig, mu>50'}
for sel in sels:
    df = {'labels' : [],'selection' : [], 'samples' :[],'samples>98odr' :[],'trig': [],'trig>98odr': [],'cf_98':[],'odr_98':[],'cf_eff':[],'odr_eff':[],'cf_chi2':[],'odr_chi2':[],'entries':[]}
    for index, (sample, props) in enumerate(full_samples.items()):
        if props['label'] == 'TT':
            continue
        df['labels'].append(props['label'])
        df['selection'].append(sel_label[sel])
        df['samples'].append(props[sel]['genwgt'].sum())
        df['samples>98odr'].append(props[sel][props[sel]['recoPFMetPt'] > props[sel]['odr_98']]['genwgt'].sum())
        df['trig'].append(props[sel][props[sel]['fired']==1]['genwgt'].sum())
        df['trig>98odr'].append(props[sel][(props[sel]['fired']==1) & (props[sel]['recoPFMetPt'] > props[sel]['odr_98'])]['genwgt'].sum())
        df['entries'].append(len(props[sel].index))
        df['cf_98'].append(props[sel]['cf_98'].iloc[0])
        df['odr_98'].append(props[sel]['odr_98'].iloc[0])
        df['cf_eff'].append(props[sel]['cf_eff'].iloc[0])
        df['odr_eff'].append(props[sel]['odr_eff'].iloc[0])
        df['cf_chi2'].append(props[sel]['cf_chi2'].iloc[0])
        df['odr_chi2'].append(props[sel]['odr_chi2'].iloc[0])
        print(props['label'],":",props[sel]['genwgt'].sum())
    #print(props['enrich'][props['enrich']['recoPFMetPt'] > props['odr_98']]['genwgt'].sum())
    for index, (sample, props) in enumerate(Combo_samples.items()):
        if props['label'] == 'MC':
            df['labels'].append(props['label'])
            df['selection'].append(sel_label[sel])
            df['samples'].append(props[sel]['genwgt'].sum())
            df['samples>98odr'].append(props[sel][props[sel]['recoPFMetPt'] > props[sel]['odr_98']]['genwgt'].sum())
            df['trig'].append(props[sel][props[sel]['fired']==1]['genwgt'].sum())
            df['trig>98odr'].append(props[sel][(props[sel]['fired']==1) & (props[sel]['recoPFMetPt'] > props[sel]['odr_98'])]['genwgt'].sum())
            df['entries'].append(len(props[sel].index))
            df['cf_98'].append(props[sel]['cf_98'].iloc[0])
            df['odr_98'].append(props[sel]['odr_98'].iloc[0])
            df['cf_eff'].append(props[sel]['cf_eff'].iloc[0])
            df['odr_eff'].append(props[sel]['odr_eff'].iloc[0])
            df['cf_chi2'].append(props[sel]['cf_chi2'].iloc[0])
            df['odr_chi2'].append(props[sel]['odr_chi2'].iloc[0])
            print(props['label'],":",props[sel]['genwgt'].sum())
    info = pd.DataFrame(df)
    info['pass'] = info['trig']/info['samples']
    info['pass>98'] = info['trig>98odr']/info['samples>98odr']
    print(info)

In [ ]:
df = {'labels' : [], 'samples' :[],'trig': [],'entries': []}
for index, (sample, props) in enumerate(Combo_samples.items()):
    df['labels'].append(props['label'])
    df['samples'].append(props['enrich']['genwgt'].sum())
    df['trig'].append(props['enrich'][props['enrich']['fired']==1]['genwgt'].sum())
    df['entries'].append(len(props['enrich'].index))
info = pd.DataFrame(df)
info['pass'] = info['trig']/info['samples']
print(info)